# Sorting and filtering list view

In below example, we customize the `QSortFilterProxyModel` to write our own sort logic. We customize a `QAbstractListModel` to add edit data logic. Then we follow the standard steps:
1. create a list view instance
2. create a MyListModel instance with source data
3. create a proxy model instance with the source model
4. connect the view with proxy model

In [ ]:
import sys
from typing import List, Any

from PyQt6.QtCore import QVariant, Qt, QModelIndex, QAbstractListModel, QSortFilterProxyModel
from PyQt6.QtWidgets import QListView, QSpinBox, QStyle, QStyledItemDelegate, QStyleOptionProgressBar, QApplication, \
    QWidget, QVBoxLayout


class SortProxyModel(QSortFilterProxyModel):
    """
    This proxy model is responsible for sorting rows in the source model. You can notice, this model does not have the
    data(), or rowCount() method to handle the data. It sits upon another model
    """

    def lessThan(self, left_index, right_index):
        """
        compare left and right, if left is smaller than right, then return true, otherwise return false
        """

        # Get the cell value base on index
        left_val = left_index.data(Qt.ItemDataRole.DisplayRole)
        right_val = right_index.data(Qt.ItemDataRole.DisplayRole)

        # by default, all cell value in pyqt is string, so we need to convert it to int
        left_int = int(left_val)
        right_int = int(right_val)

        # return the result
        return left_int < right_int

    def filterAcceptsRow(self, src_row, src_parent):
        """
        过滤接收的行
        """

        # get the source Model
        src_model = self.sourceModel()
        # 获得当前行的索引，0为列，我们目前只有一列，所以用第一列，也就是第0列
        src_index = src_model.index(src_row, 0)

        # 获取当前索引的数据
        item_val = src_index.data(Qt.ItemDataRole.DisplayRole)
        # 转为int
        item_int = int(item_val)

        # 过滤大于等于60的数据
        return item_int >= 20


class MyListModel(QAbstractListModel):
    """
    A custom List model
    """

    def __init__(self, data: List[Any], parent=None):
        super(MyListModel, self).__init__(parent)

        # 这是数据
        self._data = data

    def rowCount(self, parent=QModelIndex()):
        """
        这个方法返回了数据的行数
        也就是有多少个条目得数据
        """

        return len(self._data)

    def data(self, index, role=Qt.ItemDataRole.DisplayRole):
        """
        Get the current index, and find the corresponding data (based on required role) in the source data, then
        return it.
        """

        # if current index is not valid (e.g. <0, > max row count), we return an empty row
        if not index.isValid() or not 0 <= index.row() < self.rowCount():
            # QVariant is an empty row
            return QVariant()

        # get current row number
        row = index.row()

        # if current role is DisplayRole, return the corresponding data
        if role == Qt.ItemDataRole.DisplayRole:
            # get the data from source with current row number
            return self._data[row]

        # if current role is editRole，return the data, so when we double-click on a cell (in edit mode),
        # we can still see the origin data, not just empty cell
        if role == Qt.ItemDataRole.EditRole:
            return self._data[row]

        # For all other roles，we return an empty QVariant
        return QVariant()

    def flags(self, index):
        """
        flag描述了view中数据项的状态信息
        """

        # 首先获取超类的flags返回值
        flag = super(MyListModel, self).flags(index)

        # 或运算，将ItemIsEditable（可编辑）标志叠加上去
        return flag | Qt.ItemFlag.ItemIsEditable

    def setData(self, index, value, role=Qt.ItemDataRole.EditRole):
        """
        This method allow view to have an edit mode, which can modify the model and data source.
        After the modification, it emit a signal dataChanged, so the view can update the interface
        """
        print('MyListModel setData', index.column(), index.row())
        # 如果当前为编辑角色
        if role == Qt.ItemDataRole.EditRole:
            # QVariant的这个方法，返回的bool类型表示这个值是否可以被转为int类型
            value_int = value if value.isDigit() else 0

            # 保存数据
            self._data[index.row()] = value_int
            # 发射数据更改信号，以便让view更新
            self.dataChanged.emit(index, index)
            # 数据是否成功更新
            return True


class SortView(QWidget):
    def __init__(self):
        super().__init__()

        layout = QVBoxLayout()
        self.setLayout(layout)

        data = [10,20,50,80,90]

        # create source model with MyListModel
        model = MyListModel(data)

        # create a proxy model
        proxy = SortProxyModel()
        # connect proxy model with source model
        proxy.setSourceModel(model)

        # Set dynamic filter
        proxy.setDynamicSortFilter(True)

        # set the column which want to use to sort
        proxy.sort(0)

        # create ListView
        view = QListView()

        # set proxy model as the model of the list view
        view.setModel(proxy)

        layout.addWidget(view)


def main():
    app = QApplication(sys.argv)
    view = SortView()
    view.show()

    sys.exit(app.exec())


if __name__ == "__main__":
    main()
